# Lab 4

In the previous labs, we perform mortality prediction using DNN and CNN. However, to deal with sequential data, the most commonly used architecture is actually recurrent neural network (RNN). This lab introduces you to the motivation, implementation, and some commonly used RNN models. Let us get started!

Table of Contents:
- Motivation
- Implementation
- Modern RNN Models
- Assignment

Some contents of this lab are adapted from [Dive into Deep Learning](https://d2l.ai) and [Official PyTorch Tutorials](https://pytorch.org/tutorials/).

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [2]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
DATA_PATH = "../LAB4-lib/data"
assert os.path.isdir(DATA_PATH)
!ls {DATA_PATH}

data.csv


## 1. Motivation

While CNNs can efficiently process spatial information, recurrent neural networks (RNNs) are designed to better handle sequential information. RNNs introduce state variables to store past information, together with the current inputs, to determine the current outputs.

That is, let us say we have a sequence $x_1, x_2, \dots, x_t$. RNN attempts to model the conditional probability: $P(x_t \mid x_1, x_2, \dots, x_{t-1})$. More specifically, RNN leverages a hidden state variable  that stores the sequence information up to time step $t−1$:

$$P(x_t \mid x_{1}, x_{2}, \ldots, x_{t-1}) \approx P(x_t \mid h_{t-1}),$$

where $h_{t-1}$ is the hidden state variable. In general, the hidden state at any time step $t$ could be computed based on both the current input $x_t$ and the previous hidden state $h_{t-1}$:

$$h_t = f(x_{t}, h_{t-1}).$$ 

## 2. Implementation

Assume that we have a minibatch of inputs $\mathbf{X}_t \in \mathbb{R}^{n \times d}$ at time step $t$. In other words, for a minibatch of $n$ sequence examples, each row of $\mathbf{X}_t$ corresponds to one example at time step $t$ from the sequence. Next, denote by $\mathbf{H}_t \in \mathbb{R}^{n \times h}$ the hidden variable of time step $t$. Further, we save the hidden variable $\mathbf{H}_{t-1}$ from the previous time step and introduce a new weight parameter $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$ to describe how to use the hidden variable of the previous time step in the current time step. Specifically, the calculation of the hidden variable of the current time step is determined by the input of the current time step together with the hidden variable of the previous time step:

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$

From the relationship between hidden variables $\mathbf{H}_{t}$ and $\mathbf{H}_{t-1}$ of adjacent time steps, we know that these variables captured and retained the sequence’s historical information up to their current time step, just like the state or memory of the neural network’s current time step. Therefore, such a hidden variable is called a hidden state. Since the hidden state uses the same definition of the previous time step in the current time step, the computation of $\mathbf{H}_{t}$ is recurrent. Hence, neural networks with hidden states based on recurrent computation are named recurrent neural networks. Layers that perform the computation of $\mathbf{H}_{t}$ in RNNs are called recurrent layers.

For time step $t$, the output of the output layer is similar to the computation in the MLP:

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

Parameters of the RNN include the weights $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}, \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$, and the bias $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$ of the hidden layer, together with the weights $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ and the bias $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ of the output layer. It is worth mentioning that even at different time steps, RNNs always use these model parameters. Therefore, the parameterization cost of an RNN does not grow as the number of time steps increases.

<img src='img/rnn.svg'>

### Exercise 1 [20 points]

Implement the function calculating the current hidden state:

$$\mathbf{H}_t = \mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h.$$

Note that here `X` is the input at a sinlge time step.

In [4]:
def calculate_current_h(X, H, W_xh, W_hh, b_h):
    """
    Params:
        - X: (batch size, input dimension)
        - H: (batch size, hidden dimension)
        - W_xh: (input dimension, hidden dimension)
        - W_hh: (hidden dimension, hidden dimension)
        - b_h: (1, hidden dimension)
    """
    H_t = torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h
    return H_t

In [5]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

torch.manual_seed(42)

n, d, h = 2, 8, 4 # batch size, input dimension, hidden dimension
X, W_xh = torch.normal(0, 1, (n, d)), torch.normal(0, 1, (d, h))
H, W_hh = torch.normal(0, 1, (n, h)), torch.normal(0, 1, (h, h))
b_h = torch.normal(0, 1, (1, h))
assert torch.allclose(calculate_current_h(X, H, W_xh, W_hh, b_h),
                      torch.tensor([[ 5.1226, -0.6884,  3.1821,  6.6513],
                                    [-0.6077,  2.2313, -1.4812,  0.4403]]), rtol=1e-2)



### Exercise 2 [20 points]

Call the previous implemented `calculate_current_h` recursively to calculate the final hidden state.

Note that here `inputs` is the entire sequence.

In [6]:
def rnn(inputs, state, W_xh, W_hh, b_h):
    """
    Params:
        - inputs: (batch size, sequence length, input dimension)
        - state: (batch size, hidden dimension)
        - W_xh: (input dimension, hidden dimension)
        - W_hh: (hidden dimension, hidden dimension)
        - b_h: (1, hidden dimension)
    """
    batch_size, seq_len, input_dim = inputs.shape
    hidden_dim = state.shape[1]
    
    H_t = state
    
    for t in range(seq_len):
        X_t = inputs[:, t, :]
        H_t = calculate_current_h(X_t, H_t, W_xh, W_hh, b_h)
    
    return H_t

In [7]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

torch.manual_seed(42)

n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

inputs = torch.normal(0, 1, (n, t, d))
initial_state  = torch.normal(0, 1, (n, h))

W_xh = torch.normal(0, 1, (d, h))
W_hh = torch.normal(0, 1, (h, h))
b_h = torch.normal(0, 1, (1, h))
assert torch.allclose(rnn(inputs, initial_state, W_xh, W_hh, b_h),
                      torch.tensor([[ -46.5894,  -61.1859,   14.3644,   56.4997],
                                    [ 166.5581,   84.0468,  -24.5756, -149.2971]]), rtol=1e-2)



## 3. Modern RNN Models

We have introduced the basics of RNNs, which can better handle sequence data. However, such techniques may not be sufficient for practitioners when they face a wide range of sequence learning problems nowadays.

For instance, a notable issue in practice is the numerical instability of RNNs. Although there are several implementation tricks such as gradient clipping, this issue can be alleviated further with more sophisticated designs of sequence models. Specifically, gated RNNs are much more common in practice. In this section, we will introduce you two of such widely-used networks, namely long short-term memory (LSTM) and gated recurrent units (GRUs).

Here, we will only cover the basic concepts of GRU and LSTM. We won't discuss the architecture in details. If you are intereseted, you can refer to this [link](https://d2l.ai/chapter_recurrent-modern/index.html).

### 3.1 Gated Recurrent Units (GRU)

The key distinction between vanilla RNNs and GRUs is that the latter support gating of the hidden state. This means that we have dedicated mechanisms for when a hidden state should be updated and also when it should be reset. These mechanisms are learned and the network can selectively keep or forget information. For instance, if the first token is of great importance we will learn not to update the hidden state after the first observation. Likewise, we will learn to skip irrelevant temporary observations.

<img src='img/gru.svg'>

As shown in the figure above, the reset gates help capture short-term dependencies in sequences. And the update gates help capture long-term dependencies in sequences.

To initialize a [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU) layer in PyTorch, try the following snippet:

In [8]:
n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

# If batch_first=True, then the input and output tensors are 
# provided as (batch, seq, feature) instead of (seq, batch, feature).
rnn = nn.GRU(input_size=d, hidden_size=h, batch_first=True)

inputs = torch.randn(n, t, d)
h0 = torch.randn(1, n, h) # the first dimension is the number of RNN layers (default value is 1)

output, hn = rnn(inputs, h0)
print('output:', output.shape)
print('hn:', hn.shape)

output: torch.Size([2, 5, 4])
hn: torch.Size([1, 2, 4])


### 3.2 Long Short-Term Memory (LSTM)

The challenge to address long-term information preservation and short-term input skipping in latent variable models has existed for a long time. One of the earliest approaches to address this was the long short-term memory (LSTM) ([Hochreiter & Schmidhuber, 1997](https://direct.mit.edu/neco/article/9/8/1735/6109/Long-Short-Term-Memory)). It shares many of the properties of the GRU. Interestingly, LSTMs have a slightly more complex design than GRUs but predates GRUs by almost two decades.

LSTM introduces a memory cell (or cell for short) that has the same shape as the hidden state, engineered to record additional information. To control the memory cell we need a number of gates. One gate is needed to read out the entries from the cell. We will refer to this as the output gate. A second gate is needed to decide when to read data into the cell. We refer to this as the input gate. Last, we need a mechanism to reset the content of the cell, governed by a forget gate. The motivation for such a design is the same as that of GRUs, namely to be able to decide when to remember and when to ignore inputs in the hidden state via a dedicated mechanism.

<img src='img/lstm.svg'>

To initialize a [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer in PyTorch, try the following snippet:

In [9]:
n, t, d, h = 2, 5, 8, 4 # batch size, sequence length, input dimension, hidden dimension

# If batch_first=True, then the input and output tensors are 
# provided as (batch, seq, feature) instead of (seq, batch, feature).
rnn = nn.LSTM(input_size=d, hidden_size=h, batch_first=True)

inputs = torch.randn(n, t, d)
h0 = torch.randn(1, n, h) # the first dimension is the number of RNN layers (default value is 1)
c0 = torch.randn(1, n, h)

output, (hn, cn) = rnn(inputs, (h0, c0))
print('output:', output.shape)
print('hn:', hn.shape)
print('cn:', cn.shape)

output: torch.Size([2, 5, 4])
hn: torch.Size([1, 2, 4])
cn: torch.Size([1, 2, 4])


## Assignment [60 points]

In this assignment, you will use [MIMIC-III Demo](https://physionet.org/content/mimiciii-demo/) dataset, which contains all intensive care unit (ICU) stays for 100 patients. The task is Mortality Prediction.

### Load Data

In the previous lab, we have preprocessed the data. Thus, for this lab, we will directly use the processed data.

In [10]:
!ls {DATA_PATH}

data.csv


Here are the helper fuctions and CustomDataset from the previous lab. 

We will use the entire patient visit instead of only the last visit.

Note that in this lab, we **do not** need to exclude patients with only one visit.

In [11]:
# two helper functions

TOTAL_NUM_CODES = 271


def read_csv(filename):
    """ reading csv from filename """
    data = []
    with open(filename, "r") as file:
        csv_reader = csv.DictReader(file, delimiter=',')
        for row in csv_reader:
            data.append(row)
    header = list(data[0].keys())
    return header, data


def to_one_hot(label, num_class):
    """ convert to one hot label """
    one_hot_label = [0] * num_class
    for i in label:
        one_hot_label[i] = 1
    return one_hot_label

In [12]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self):
        # read the csv
        self._df = pd.read_csv(f'{DATA_PATH}/data.csv')
        # split diagnosis code index by ';' and convert it to integer
        self._df.icd9 = self._df.icd9.apply(lambda x: [int(i) for i in x.split(';')])
        # build data dict
        self._build_data_dict()
        # a list of subject ids
        self._subj_ids = list(self._data.keys())
        # sort the subject ids to maintain a fixed order
        self._subj_ids.sort()
    
    def _build_data_dict(self):
        """ 
        build SUBJECT_ID to ADMISSION dict
            - subject_id
                - icd9: a list of ICD9 code index
                - mortality: 0/1 morality label
        """
        dict_data = {}
        df = self._df.groupby('subject_id').agg({'mortality': lambda x: x.iloc[0], 'icd9': list}).reset_index()
        for idx, row in df.iterrows():
            subj_id = row.subject_id
            dict_data[subj_id] = {}
            dict_data[subj_id]['icd9'] = row.icd9
            dict_data[subj_id]['mortality'] = row.mortality
        self._data = dict_data
    
    def __len__(self):
        """ return the number of samples (i.e. patients). """
        return len(self._subj_ids)
    
    def __getitem__(self, index):
        """ generates one sample of data. """
        # obtain the subject id
        subj_id = self._subj_ids[index]
        # obtain the data dict by subject id
        data = self._data[subj_id]
        # convert last admission's diagnosis code index to one hot
        x = torch.tensor([to_one_hot(visit, TOTAL_NUM_CODES) for visit in data['icd9']], dtype=torch.float32)
        # mortality label
        y = torch.tensor(data['mortality'], dtype=torch.float32)
        return x, y

In [13]:
dataset = CustomDataset()
print('Size of dataset:', len(dataset))

Size of dataset: 99


In [14]:
from torch.utils.data.dataset import random_split


split = int(len(dataset)*0.7)

lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 69
Length of test dataset: 30


Here is an example of $x$, and $y$. 

In [15]:
x, y = train_dataset[0]
print(f'Example x (shape {x.shape}):\n', x)
print(f'Example y:\n', y)

Example x (shape torch.Size([1, 271])):
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

We can see that $x$ is of shape $(2, 271)$, which means there are $271$ diagnosis codes in total, and this patient has two visits. It is in one-hot format. A $1$ in position $i$ means that diagnosis code of index $i$ appears in the that visit.

And $y$ is either $0$ or $1$.

### Padding [20 points]

In the previous lab, we implement a collate function `collate_fn()` to pad the sequence into the same length. For RNN, we will do something similar.

Moreover, we will keep a separate variable storing the length of each sequence. Later, we will use this length variable to select the mask out padding visits.

In [16]:
def collate_fn(data):
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, total # diagnosis codes). Further, you need to store 
        the true length of each sequence into l.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patients, total # diagnosis codes, max # visits) of type torch.float
        y: a tensor of shape (# patients) of type torch.float
        l: a tensor of shape (# patients) of type torch.long
        
    Note that you can obtains the list of diagnosis codes and the list of mortality labels
        using: `sequences, labels = zip(*data)`
    """

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [patient.shape[0] for patient in sequences]
    total_num_codes = sequences[0].shape[1]

    max_num_visits = max(num_visits)
    
    x = torch.zeros((num_patients, max_num_visits, total_num_codes), dtype=torch.float)
    l = torch.tensor(num_visits, dtype=torch.long)
    
    for i, patient in enumerate(sequences):
        x[i, :num_visits[i], :] = patient
    
    return x, y, l

In [17]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from torch.utils.data import DataLoader


loader = DataLoader(train_dataset, batch_size=4, collate_fn=collate_fn)
loader_iter = iter(loader)
x, y, l = next(loader_iter)

print(x.shape)
print(y.shape)
print(l)

assert x.dtype == torch.float
assert y.dtype == torch.float
assert l.dtype == torch.long

assert x.shape[0] == 4
assert x.shape[-1] == 271
assert y.shape == (4,)
assert l.shape == (4,)

for i in range(4):
    real_x, real_y = train_dataset[i]
    assert len(real_x) == l[i]
    for j in range(real_x.shape[0]):
        visit = real_x[j]
        got = x[i, j, :]
        assert all(visit == got)
        assert real_y == y[i]



torch.Size([4, 3, 271])
torch.Size([4])
tensor([1, 3, 1, 1])


### Data Loader

Now, we can load the dataset into the data loader.

In [18]:
from torch.utils.data import DataLoader

# how many samples per batch to load
batch_size = 4

# prepare dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

print("# of train batches:", len(train_loader))
print("# of test batches:", len(test_loader))

# of train batches: 18
# of test batches: 8


In [19]:
train_iter = iter(train_loader)
x, y, l = next(train_iter)

print('Shape of a batch x:', x.shape)
print('Shape of a batch y:', y.shape)
print('Shape of a batch l:', l.shape)

Shape of a batch x: torch.Size([4, 1, 271])
Shape of a batch y: torch.Size([4])
Shape of a batch l: torch.Size([4])


### Build the Model

<img src='img/naive-rnn.png'>

We will construct this simple RNN structure. So each input is a one-hot vector. At the 0-th visit, this has $\boldsymbol{X}_0$, and at t-th visit, this has $\boldsymbol{X}_t$.

Each one of them will then map to a hidden state $\boldsymbol{h}_t$. The hidden state $\boldsymbol{h}_t$ can be determined by $\boldsymbol{h}_{t-1}$ and the corresponding current input $\boldsymbol{X}_t$.

Finally, once we have the $\boldsymbol{h}_T$, the hidden state of the last timestamp, then we can use this as feature vectors and train a NN to perform the classification.

Now, let us build this model. The forward steps will be:

    1. Pass the inputs through the RNN layer;
    2. Obtain the hidden state at the last visit;
    3. Pass the hidden state through the linear and activation layers.

#### Mask Selection [20 points]

Importantly, you need to use `length` to mask out the paddings in step 2.

In [20]:
def get_last_visit(hidden_states, length):
    """
    TODO: obtain the hidden state for the last true visit (not padding visits)

    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, hidden_dim)
        length: the true visit length of shape (batch_size,)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, hidden_dim)
        
    NOTE: DO NOT use for loop.
    """
    
    batch_size, num_visits, hidden_dim = hidden_states.shape
    
    # Adjust lengths to be within the range of indices
    adjusted_length = length - 1  # Adjust because lengths are 1-based indices

    # Create a tensor for batch indices
    batch_indices = torch.arange(batch_size, device=hidden_states.device)

    # Use advanced indexing to get the last hidden state
    last_hidden_state = hidden_states[batch_indices, adjusted_length]
    
    return last_hidden_state

In [21]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

max_num_visits = 10
batch_size = 16
hidden_dim = 100

hidden_states = torch.randn((batch_size, max_num_visits, hidden_dim))
lengths = torch.tensor([random.randint(1, max_num_visits) for _ in range(batch_size)])
out = get_last_visit(hidden_states, lengths)

assert out.shape == (batch_size, hidden_dim)

for b in range(batch_size):
    last_h = 0
    last_h = hidden_states[b, lengths[b] - 1]
    assert torch.allclose(out[b], last_h, atol=1e-4), \
    "The last visit's hidden state of %d-th visit of the %d-th patient is wrong. "%(v,b) +\
    "Expect {} Got {} with your get_last_visit".format(last_h, out[b])



#### Build NaiveRNN [20 points]

In [22]:
import torch.nn.utils.rnn as rnn_utils

class NaiveRNN(nn.Module):
    
    """
    TODO: implement the naive RNN model above.
    """
    
    def __init__(self):
        super().__init__()
        """
        TODO: 
            1. Define the RNN using `nn.GRU()`; Set `input_size` to `TOTAL_NUM_CODES`.
               Set `hidden_size` to 32. Set `batch_first` to True.
            2. Define the linear layers using `nn.Linear()`; Set `output_size` to 1.
            3. Define the final activation layer using `nn.Sigmoid().
        """
        
        # Define the RNN using `nn.GRU()`
        self.rnn = nn.GRU(input_size=TOTAL_NUM_CODES, hidden_size=32, batch_first=True)
        
        # Define the linear layer using `nn.Linear()`
        self.fc = nn.Linear(32, 1)
        
        # Define the final activation layer using `nn.Sigmoid()`
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, length):
        """
        TODO:
            1. Pass the inputs through the RNN layer;
            2. Obtain the hidden state at the last visit.
               Use `get_last_visit()`;
            3. Pass the hidden state through the linear and activation layers.
            
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            length: the true visit length of shape (batch_size,)

        Outputs:
            probs: probabilities of shape (batch_size)
        """
        
        # Pass the inputs through the RNN layer
        x_packed = rnn_utils.pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
        _, h_n = self.rnn(x_packed)
        
        # h_n contains the hidden state for the last non-padded visit
        # Pass the hidden state through the linear and activation layers
        out = self.fc(h_n[-1])
        probs = self.sigmoid(out).squeeze()
        
        return probs
    

# load the model here
model = NaiveRNN()
model

NaiveRNN(
  (rnn): GRU(271, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [23]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

layers_to_check = [nn.GRU, nn.Linear, nn.Sigmoid]
for layer_to_check in layers_to_check:
    no_layer = True
    for child in model.children():
        for layer in child.modules():
            if(isinstance(layer, layer_to_check)):
                no_layer = False
    assert not no_layer, "{} is missing in your RNN".format(layer_to_check)

loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(loader)
x, y, l = next(loader_iter)
model_output = model(x, l)
assert model_output.shape == (10,), "Your RNN's output shape is {}, expect {}".format(model_output.shape, (10,))



### Train the Network

In this step, you will train the CNN model.

In [24]:
# Use Binary Cross Entropy as the loss function (`nn.BCELoss`)
# Use Adam as the optimizer (`torch.optim.Adam`)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Now we can train the model.

In [25]:
# from sklearn.metrics import *

#input: Y_score,Y_pred,Y_true
#output: accuracy, auc, precision, recall, f1-score
# def classification_metrics(Y_score, Y_pred, Y_true):
#     acc, auc, precision, recall, f1score = accuracy_score(Y_true, Y_pred), \
#                                            roc_auc_score(Y_true, Y_score), \
#                                            precision_score(Y_true, Y_pred), \
#                                            recall_score(Y_true, Y_pred), \
#                                            f1_score(Y_true, Y_pred)
#     return acc, auc, precision, recall, f1score


#input: model, loader
# def evaluate(model, loader):
#     model.eval()
#     all_y_true = torch.LongTensor()
#     all_y_pred = torch.LongTensor()
#     all_y_score = torch.FloatTensor()
#     for x, y, l in loader:
#         # pass the input through the model
#         y_hat = model(x, l)
#         # convert shape from [batch size, 1] to [batch size]
#         y_hat = y_hat.view(y_hat.shape[0])
#         y_pred = (y_hat > 0.5).type(torch.float)
#         all_y_true = torch.cat((all_y_true, y.to('cpu')), dim=0)
#         all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu')), dim=0)
#         all_y_score = torch.cat((all_y_score,  y_hat.to('cpu')), dim=0)
        
#     acc, auc, precision, recall, f1 = classification_metrics(all_y_score.detach().numpy(), 
#                                                              all_y_pred.detach().numpy(), 
#                                                              all_y_true.detach().numpy())
#     print(f"acc: {acc:.3f}, auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
#     return

In [26]:
# print("model perfomance before training:")
# evaluate(model, train_loader)
# evaluate(model, test_loader)

In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import numpy as np

# Input: Y_score, Y_pred, Y_true
# Output: accuracy, auc, precision, recall, f1-score
def classification_metrics(Y_score, Y_pred, Y_true):
    acc = accuracy_score(Y_true, Y_pred)
    auc = roc_auc_score(Y_true, Y_score)
    precision = precision_score(Y_true, Y_pred)
    recall = recall_score(Y_true, Y_pred)
    f1 = f1_score(Y_true, Y_pred)
    return acc, auc, precision, recall, f1

# Input: model, loader
def evaluate(model, loader):
    model.eval()
    all_y_true = []
    all_y_pred = []
    all_y_score = []
    
    with torch.no_grad():  # Disable gradient computation
        for x, y, l in loader:
            # Pass the input through the model
            y_hat = model(x, l).squeeze()
            # Convert shape from [batch size, 1] to [batch size]
            y_pred = (y_hat > 0.5).type(torch.float)
            
            all_y_true.extend(y.cpu().numpy().flatten())
            all_y_pred.extend(y_pred.cpu().numpy().flatten())
            all_y_score.extend(y_hat.cpu().detach().numpy().flatten())
            
    acc, auc, precision, recall, f1 = classification_metrics(
        np.array(all_y_score), 
        np.array(all_y_pred), 
        np.array(all_y_true)
    )
    print(f"acc: {acc:.3f}, auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
    return acc, auc, precision, recall, f1

# Check model performance before training
print("Model performance before training:")
evaluate(model, train_loader)
evaluate(model, test_loader)


Model performance before training:
acc: 0.725, auc: 0.516, precision: 0.731, recall: 0.980, f1: 0.838
acc: 0.667, auc: 0.730, precision: 0.667, recall: 1.000, f1: 0.800


(0.6666666666666666, 0.73, 0.6666666666666666, 1.0, 0.8)

In [28]:
# number of epochs to train the model
# feel free to change this
# n_epochs = 15

# prep model for training
# model.train()

# for epoch in range(n_epochs):
    
#     train_loss = 0
#     for x, y, l in train_loader:
#         """ Step 1. clear gradients """
#         optimizer.zero_grad()
#         """  Step 2. perform forward pass using `model`, save the output to y_hat """
#         y_hat = model(x, l)
#         """ Step 3. calculate the loss using `criterion`, save the output to loss. """
#         # convert shape from [batch size, 1] to [batch size]
#         y_hat = y_hat.view(y_hat.shape[0])
#         loss = criterion(y_hat, y)
#         """ Step 4. backward pass """
#         loss.backward()
#         """ Step 5. optimization """
#         optimizer.step()
#         """ Step 6. record loss """
#         train_loss += loss.item()
        
#     train_loss = train_loss / len(train_loader)
#     print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
#     evaluate(model, train_loader)
#     evaluate(model, test_loader)

In [29]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)

In [30]:
# number of epochs to train the model
n_epochs = 15

# prep model for training
model.train()

for epoch in range(n_epochs):
    
    train_loss = 0
    for x, y, l in train_loader:
        """ Step 1. clear gradients """
        optimizer.zero_grad()
        """  Step 2. perform forward pass using `model`, save the output to y_hat """
        y_hat = model(x, l)
        print(f"y_hat shape: {y_hat.shape}")  # Debugging line
        """ Step 3. calculate the loss using `criterion`, save the output to loss. """
        # convert shape from [batch size, 1] to [batch size]
        y_hat = y_hat.view(-1)
        loss = criterion(y_hat, y)
        """ Step 4. backward pass """
        loss.backward()
        """ Step 5. optimization """
        optimizer.step()
        """ Step 6. record loss """
        train_loss += loss.item()
        
    train_loss = train_loss / len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    evaluate(model, train_loader)
    evaluate(model, test_loader)


y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
Epoch: 1 	Training Loss: 0.649239
acc: 0.719, auc: 0.591, precision: 0.719, recall: 1.000, f1: 0.836
acc: 0.812, auc: 1.000, precision: 0.812, recall: 1.000, f1: 0.897
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
Epoch: 2 	Training Loss: 0.636491
acc: 0.703, auc: 0.643, precision: 0.703, recall: 1.000, f1: 0.826
acc: 0.812, auc: 1.000, precision: 0.812, recall: 1.000, f1: 0.897
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
Epoch: 3 	Training Loss: 0.612552
acc: 0.734, auc: 0.721, precision: 0.734, recall: 1.000, f1: 0.847
acc: 0.812, auc: 0.974, precision: 0.812, recall: 1.000, f1: 0.897
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
y_hat shape: torch.Size([16])
Epoch: 4 	Traini

The result is bad due to very limited data. The model overfits the training data very fast.

You are encouraged to try this on the whole MIMIC-III dataset. The result will be much more promising!